In [24]:
import pandas as pd
import numpy as np
import yake as yk

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string

from joblib import dump, load

## Recommending articles using Doc2Vec

### Approach:
1. **Train a Doc2Vec Model:**
   - Utilize Doc2Vec to transform entire articles into vector representations.
   - Train the model to capture contextual information and semantic meaning of each document.

2. **Compute Similarity:**
   - Classify the article provided by the user.
   - Measure similarity between the provided article and the other articles based on the cosine similarity of their Doc2Vec vectors.
   - Cosine similarity provides a metric for how closely the semantic meanings align.

3. **Recommend articles:**
   - Provide a ranking of the articles based on the cosine similarity value.
   - Recommend the top n articles.    
     

4. **Benefits of Doc2Vec:**
   - Doc2Vec models excel in capturing contextual nuances and semantic relationships within entire documents.

This approach enables the recommendation of article based on their semantic similarity.  
  
We use the BBC_News_Test.csv file to simulate non classified articles provided by the user.  
The BBC_News_Train.csv file is used as a database of classified articles to be recommended.


## Train a Doc2Vec Model

Doc2Vec is a Model that represents each Document as a Vector. It usually outperforms Word2Vec model.  
  
Corresponding research paper: https://cs.stanford.edu/~quocle/paragraph_vector.pdf

### Articles vectorization

First import preprocessed data (punctuation and stop words removed + lemmatization)

In [25]:
df = pd.read_csv("./data/BBC_News_Train_PREPROCESSED.csv")
documents = df['Text']
df.head(10)

,Unnamed: 0,ArticleId,Text,Category
0,0,1833,worldcom exboss launch defence lawyer defendin...,business
1,1,154,german business confidence slide german busine...,business
2,2,1101,bbc poll indicates economic gloom citizen majo...,business
3,3,1976,lifestyle governs mobile choice faster better ...,tech
4,4,917,enron boss 168m payout eighteen former enron d...,business
5,5,1582,howard truanted play snooker conservative lead...,politics
6,6,651,wale silent grand slam talk rhys williams say ...,sport
7,7,1797,french honour director parker british film dir...,entertainment
8,8,2034,car giant hit mercedes slump slump profitabili...,business
9,9,1866,fockers fuel festive film chart comedy meet fo...,entertainment


Train the Doc2Vec model (cf documentation: https://radimrehurek.com/gensim/models/doc2vec.html)

In [26]:
# Be careful: the words property should be a list of strings, not a string.
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(documents)]

# Training the Doc2Vec model
vector_size = 100
model = Doc2Vec(vector_size=vector_size, window=5, min_count=1, workers=4, epochs=10)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Retrieving vectors for existing documents
existing_document_index = 0
# the dv object contains the paragraph vectors learned from the training data
existing_document_vector = model.dv[existing_document_index]
existing_document_vector

array([-0.07857528,  0.33182847,  0.20468056,  0.24599928,  0.16273426,
       -0.23477042,  0.14756352,  0.39796862, -0.52342683, -0.112628  ,
       -0.30007166, -0.25608447, -0.11764175, -0.0601214 , -0.12886949,
       -0.05899862,  0.06686479,  0.16550331, -0.00380085, -0.16142464,
        0.21397388,  0.0026129 ,  0.5454849 , -0.24973461,  0.147678  ,
        0.13634455, -0.1579878 , -0.18515743, -0.3700711 , -0.01661292,
        0.40134263,  0.11034008, -0.09458878, -0.22046015, -0.07302599,
        0.19916329,  0.2627833 , -0.1730797 ,  0.19365929, -0.38769025,
        0.30916998, -0.18601434, -0.58647406, -0.54304886,  0.26618704,
        0.09811361,  0.01494919,  0.17972212,  0.00346636,  0.2973383 ,
        0.16502751, -0.029097  , -0.0348953 ,  0.05482029, -0.30947557,
        0.37770572,  0.03832432, -0.12182017, -0.31786442,  0.26333216,
       -0.1256457 ,  0.34253162, -0.40930095,  0.24637397,  0.31226024,
        0.50045264,  0.3787878 ,  0.24426447, -0.28316435,  0.41

To vectorize a new document, we should execute : 

`new_document = ['new', 'document', 'to', 'vectorize']`<br>
`vector = model.infer_vector(new_document)`

Here's an example.

In [27]:
new_doc = ["let's", "test", "the", "vectorizer"]
model.infer_vector(new_doc)

array([-9.07147210e-03,  2.32103262e-02,  2.71451473e-03,  1.75406355e-02,
        9.05170944e-03, -4.89571318e-03, -3.74486251e-03,  4.19744588e-02,
       -1.42156035e-02, -1.17760315e-03, -2.15377267e-02, -3.63908447e-02,
       -1.03198430e-02,  9.02685337e-03, -1.57757495e-02, -5.07096052e-02,
        1.13303177e-02, -3.49301621e-02,  1.78728458e-02, -1.94771569e-02,
        7.28532160e-03, -1.35415886e-03,  3.36161144e-02,  4.83258953e-03,
        8.31873604e-06,  3.26711126e-03, -1.47200711e-02,  9.32923518e-04,
       -1.60396043e-02, -5.20296302e-03,  2.34834701e-02,  8.28735251e-03,
       -9.33518726e-03,  2.06272509e-02,  3.01440665e-03,  1.38355680e-02,
        3.21960938e-03, -2.31293254e-02, -5.45547437e-03,  2.19131610e-03,
        4.80396533e-03, -4.23636287e-02, -3.47120576e-02, -2.31794175e-02,
        1.29481908e-02, -1.44147016e-02, -3.91755057e-05,  5.85555006e-03,
        1.45233714e-03,  3.26908119e-02, -1.97621994e-03, -1.34854997e-02,
        8.43547191e-03,  

Now, we want to vectorize each document of the dataset.

In [28]:
# Get vectors
df["vectors"] = df.apply(lambda row: model.infer_vector(row["Text"].split()), axis=1)
df.head()

,Unnamed: 0,ArticleId,Text,Category,vectors
0,0,1833,worldcom exboss launch defence lawyer defendin...,business,"[-0.022909092, 0.63232017, 0.53777283, 0.32013..."
1,1,154,german business confidence slide german busine...,business,"[0.025755063, 0.117647626, 0.13063857, 0.23446..."
2,2,1101,bbc poll indicates economic gloom citizen majo...,business,"[-0.04641482, -0.26560748, 0.49538204, 0.45341..."
3,3,1976,lifestyle governs mobile choice faster better ...,tech,"[-0.35049495, -0.39567512, 0.9580003, 0.925127..."
4,4,917,enron boss 168m payout eighteen former enron d...,business,"[-0.12963559, 0.5346129, 0.407631, 0.4868632, ..."


### Add articles without preprocessing

To be fed to the keywords extractor that will handle this task.

In [29]:
#df.set_index(["Unnamed: 0"],inplace=True)
# Get the initial DataFrame with the articles before processing
articles = pd.read_csv("./data/BBC_News_Train.csv")

# add articles to the vectorized df
df["Article"] = articles["Text"]

In [30]:
df["Article"].loc[0]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

### Keywords extraction

We extract the key words (using yake keywords extractor: https://github.com/LIAAD/yake) as an evaluation material for the recommendation.

In [31]:
kw_extractor = yk.KeywordExtractor(top=10, n=2)

In [32]:
def extract_keywords_only(article):
    keywords = []
    keywords_and_weight = kw_extractor.extract_keywords(article)

    for couple in keywords_and_weight :
        keywords.append(couple[0])
    
    return keywords

In [33]:
df["Keywords"] = df["Article"].apply(lambda article: extract_keywords_only(article))

In [34]:
df.head(3)

,Unnamed: 0,ArticleId,Text,Category,vectors,Article,Keywords
0,0,1833,worldcom exboss launch defence lawyer defendin...,business,"[-0.022909092, 0.63232017, 0.53777283, 0.32013...",worldcom ex-boss launches defence lawyers defe...,"[ex-boss launches, launches defence, worldcom,..."
1,1,154,german business confidence slide german busine...,business,"[0.025755063, 0.117647626, 0.13063857, 0.23446...",german business confidence slides german busin...,"[knocking hopes, speedy recovery, business con..."
2,2,1101,bbc poll indicates economic gloom citizen majo...,business,"[-0.04641482, -0.26560748, 0.49538204, 0.45341...",bbc poll indicates economic gloom citizens in ...,"[world economy, world, countries, economy, glo..."


### Export the dataset

For easier use of the preprocessed data later...

In [35]:
dump(df, "./assets/BBC_News_Preprocessed.joblib")
dump(model, "./assets/model_Doc2Vec.joblib")

['./assets/model_Doc2Vec.joblib']

### Compute most similiar article 

Function to preprocess an article.

In [36]:
def preprocess_article(article):
    # Tokenize the article
    words = article.split()

    # Remove punctuation
    punctuation = string.punctuation
    words = [word for word in words if word not in punctuation]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return words

Function to compute the cosine similarity between an article and all the articles of the test dataset.

In [37]:
def compute_similarity_scores(article, articles_vec, model):
    article = preprocess_article(article)
    article_vector = model.infer_vector(article)

    vectors = articles_vec['vectors']
    i = 0
    similarity_score_dict = dict()
    for vector in vectors:

        # Reshape the vectors to be 2D arrays with a single row
        vector = vector.reshape(1, -1)
        article_vector = article_vector.reshape(1, -1)

        similarity_score = cosine_similarity(vector, article_vector)[0, 0]
        
        similarity_score_dict[i] = similarity_score
        i += 1

    return similarity_score_dict

Functions to get the top 3 most similar articles.

In [38]:
def most_similar_articles(article, category):
    articles_vec = load("./assets/BBC_News_Preprocessed.joblib")
    model = load("./assets/model_Doc2Vec.joblib")
    
    # Filter articles by the specified category
    articles_vec = articles_vec[articles_vec["Category"] == category]

    scores = compute_similarity_scores(article, articles_vec, model)
    sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

    # Get the top three keys (indices)
    top_three_keys = list(sorted_scores.keys())[:3]
    top_three_keys = articles_vec.index[top_three_keys]

    # Extract the corresponding articles
    top_three_articles = articles_vec.loc[top_three_keys, ["Article", "Category"]]  

    return top_three_articles

In [39]:
def get_recommendations(article):
    svc = load("./assets/classifier.joblib")
    category = svc.predict([article])[0]
    print(f"Initial article categorized as {category}.")

    recommendations = most_similar_articles(article, category)
    return recommendations

### Testing the model

In [40]:
articles_test = pd.read_csv("./data/BBC_News_Test.csv")
article = articles_test.iloc[1]["Text"]
print(article)

software watching while you work software that can not only monitor every keystroke and action performed at a pc but also be used as legally binding evidence of wrong-doing has been unveiled.  worries about cyber-crime and sabotage have prompted many employers to consider monitoring employees. the developers behind the system claim it is a break-through in the way data is monitored and stored. but privacy advocates are concerned by the invasive nature of such software.  the system is a joint venture between security firm 3ami and storage specialists bridgehead software. they have joined forces to create a system which can monitor computer activity  store it and retrieve disputed files within minutes. more and more firms are finding themselves in deep water as a result of data misuse. sabotage and data theft are most commonly committed from within an organisation according to the national hi-tech crime unit (nhtcu) a survey conducted on its behalf by nop found evidence that more than 80

In [41]:
get_recommendations(article)

Initial article categorized as tech.


,Article,Category
1388,software watching while you work software that...,tech
1370,warning over windows word files writing a micr...,tech
1190,rich pickings for hi-tech thieves viruses tro...,tech


## Evaluation (or alternative): keywords extraction

We tried to automate the evaluation of recommendation system using keywords extraction.   
 

For each article in the test dataset, we compare the top 3 articles recommended using cosine similarity and the top 5 articles recommended using the keywords extraction (ranking obtained using the number of keywords in common between the article provided by the user and the others).  
  

The issue with this method is that we're trying to evaluate a method (ranking according to cosine similarity) using another method (ranking according to keywords in common) that is supposed to be less efficient.  
  
This is probably a bad method for evaluation, but we let it here anyway to show out attempt to automate the evaluation process...

### Ranking based on keywords extraction

In [42]:
def rank_article_basedOn_keywords(user_article):
    df = load("./assets/BBC_News_Preprocessed.joblib")
    keywords_user_article = extract_keywords_only(user_article)

    similar_articles = {}

    for idx, article_row in df.iterrows():
        keywords_article = article_row["Keywords"]
        nombre_elements_communs = len(set(keywords_user_article) & set(keywords_article))
        similar_articles[idx] = nombre_elements_communs
        

    sorted_similar_articles = dict(sorted(similar_articles.items(), key=lambda item: item[1], reverse=True))

    # Indices of articles and number of keywords in common with the article provided by the user
    return dict(list(sorted_similar_articles.items())[:5])


In [43]:
# Get recommandation accuracy for one article
def calculate_recommendation_accuracy(recommendations_cosine, top_5_similar_articles_kw):
    # Get top 3 recommended articles indices (using cosine similarity)
    recommended_indices = recommendations_cosine.index.values.tolist()
    
    # Get top 5 recommended articles indices (using keywords)
    top_5_indices = list(top_5_similar_articles_kw.keys())
    
    # Get the intersection 
    common_indices = set(recommended_indices) & set(top_5_indices)
    
    
    # Get "accuracy": if there are 3 articles in common, accuracy is 1. If only 2, accuracy is 2/3, and so on...
    accuracy = len(common_indices) / len(recommended_indices)
    
    return accuracy

### Compute the accuracy of our recommandation system forthe hole dataset

Get test articles (provided by the user)

In [44]:
df_test = pd.read_csv("./data/BBC_News_Test.csv")
articles_test = df_test["Text"]

Get final accuracy for the whole dataset

In [45]:
accuracies = []
for article_test in articles_test :
    recommendations = get_recommendations(article=article_test)
    top_5_similar_articles_indices = rank_article_basedOn_keywords(user_article=article_test)
    accuracies.append(calculate_recommendation_accuracy(recommendations_cosine=recommendations, top_5_similar_articles_kw=top_5_similar_articles_indices))
accuracy = sum(accuracies)/len(accuracies)
print(f"Recommandation System has an accuracy of : {accuracy}")

Initial article categorized as sport.
Initial article categorized as tech.
Initial article categorized as sport.
Initial article categorized as business.
Initial article categorized as sport.
Initial article categorized as sport.
Initial article categorized as politics.
Initial article categorized as politics.
Initial article categorized as entertainment.
Initial article categorized as business.
Initial article categorized as business.
Initial article categorized as tech.
Initial article categorized as politics.
Initial article categorized as tech.
Initial article categorized as entertainment.
Initial article categorized as sport.
Initial article categorized as politics.
Initial article categorized as tech.
Initial article categorized as entertainment.
Initial article categorized as entertainment.
Initial article categorized as business.
Initial article categorized as politics.
Initial article categorized as sport.
Initial article categorized as business.
Initial article categorized as

In [46]:
accuracy

0.14376417233560068

The accuracy is low, but it doesn't really mean something...

## Manual evaluation

To evaluate the recommendation system, we provide one article non classified and take 10 articles in the dataset as the articles to be recommended.  
We individually choose 3 articles that appear to us as the most similar and see if it aligns with what the model predicted.

In [47]:
def most_similar_ten_articles(article, category):
    articles_vec = load("./assets/BBC_News_Preprocessed.joblib")
    model = load("./assets/model_Doc2Vec.joblib")

    # Filter articles by the specified category
    articles_vec = articles_vec[articles_vec["Category"] == category]
    random_articles = articles_vec.sample(n= 10)

    scores = compute_similarity_scores(article, random_articles, model)
    sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

    # Get the top three keys (indices)
    top_three_keys = list(sorted_scores.keys())[:3]
    top_three_keys = random_articles.index[top_three_keys]

    # Extract the corresponding articles
    top_three_articles = random_articles.loc[top_three_keys, ["Article", "Category"]]  

    return top_three_articles, random_articles

In [51]:
articles_test = pd.read_csv("./data/BBC_News_Test.csv")
article = articles_test.iloc[1]["Text"]

In [52]:
similiar_articles, articles = most_similar_ten_articles(article, "tech")

In [67]:
with open("./data/ten_articles.txt", "w") as f :
    for article in articles["Article"] : 
        f.write(article)
        f.write("\n")
        f.write("-"*90)
        f.write("\n")

f.close()

TypeError: TextIOWrapper.write() takes exactly one argument (3 given)